In [1]:
import pandas as pd

In [17]:
# Points of interest
AreaPOI = pd.read_csv("AreaPOI.csv", usecols=["Name_lang", "Description_lang", "Pos[0]", "Pos[1]", "ContinentID", "AreaID"])

# Continents (and instances?)
# Map.MapDescription0_lang is for the Horde, and MapDescription1_lang is for the Alliance
# Map.ExpansionID is expansion ID (e.g. 7 is BfA)
Map = pd.read_csv("Map.csv", usecols=["ID", "MapName_lang"])

# Zones (and other locations)
AreaTable = pd.read_csv("AreaTable.csv", usecols=["ID", "AreaName_lang"])

In [45]:
AreaPOI_joined = (pd.merge(AreaPOI, Map, how="left", left_on="ContinentID", right_on="ID", suffixes=["", "_Map"])
                  .drop(labels=["ID"], axis=1))
AreaPOI_joined = (pd.merge(AreaPOI_joined, AreaTable, how="left", left_on="AreaID", right_on="ID", suffixes=["", "_AreaTable"])
                  .drop(labels=["ID", "AreaID"], axis=1))
AreaPOI_joined.Name_lang.str.startswith("[DEPRECATED]", na=False)
AreaPOI_joined["Origin"] = "AreaPOI (Points of Interest)"

In [31]:
# Portal entrances and exits
WaypointNode = pd.read_csv("WaypointNode.csv", usecols=["Name_lang", "Field_8_2_0_30080_005", "SafeLocID"]).rename(columns={"Field_8_2_0_30080_005": "NodeType"})
# Portal coordinates
WaypointSafeLocs = pd.read_csv("WaypointSafeLocs.csv", usecols=["ID", "Pos[0]", "Pos[1]", "MapID"])

## Commence processing
WaypointSafeLocs_joined = (pd.merge(WaypointSafeLocs, Map, how="left", left_on="MapID", right_on="ID", suffixes=["", "_Map"])
                               .drop(labels=["ID_Map"], axis=1))
WaypointNode = WaypointNode[WaypointNode.NodeType == 2] # Keep only portal exits
WaypointNode_joined = (pd.merge(WaypointNode, WaypointSafeLocs_joined, how="left", left_on="SafeLocID", right_on="ID")
                       .drop(labels=["SafeLocID", "ID", "NodeType"], axis=1)
                       .rename(columns={"MapID": "ContinentID"}))
WaypointNode_joined = WaypointNode_joined[~WaypointNode_joined.Name_lang.str.startswith("Take the")]
WaypointNode_joined["Origin"] = "WaypointNode (Portals) - Exits only"
WaypointNode_joined

,Name_lang,Pos[0],Pos[1],ContinentID,MapName_lang,Origin
0,"Wizard's Sanctum, Portal to Jade Forest",-307.16,-1772.890,870,Pandaria,WaypointNode (Portals) - Exits only
1,Uldum,-9443.31,-958.360,1,Kalimdor,WaypointNode (Portals) - Exits only
2,Mount Hyjal,5534.08,-3624.690,1,Kalimdor,WaypointNode (Portals) - Exits only
3,"Stormwind, The Eastern Earthshrine",-8209.33,428.635,0,Eastern Kingdoms,WaypointNode (Portals) - Exits only
4,Twilight Highlands,-4890.07,-6626.740,0,Eastern Kingdoms,WaypointNode (Portals) - Exits only
...,...,...,...,...,...,...
99,Oribos,-9041.61,917.320,0,Eastern Kingdoms,WaypointNode (Portals) - Exits only
100,Oribos,1439.93,-4484.690,1,Kalimdor,WaypointNode (Portals) - Exits only
101,Oribos - Shadowlands,-1834.21,1542.270,2222,The Shadowlands,WaypointNode (Portals) - Exits only
102,Maldraxxus,-2983.85,-4501.560,2222,The Shadowlands,WaypointNode (Portals) - Exits only


In [30]:
# TaxiNodes: Flight points
TaxiNodes = pd.read_csv("TaxiNodes.csv", usecols=["Name_lang", "Pos[0]", "Pos[1]", "ContinentID"])
TaxiNodes_joined = (pd.merge(TaxiNodes, Map, how="left", left_on="ContinentID", right_on="ID", suffixes=["", "_Map"])
                   .drop(labels=["ID"], axis=1))
TaxiNodes_joined = TaxiNodes_joined[~TaxiNodes_joined.Name_lang.str.startswith("Quest")]
TaxiNodes_joined["Origin"] = "TaxiNodes (Flight points table)"
TaxiNodes_joined

,Name_lang,Pos[0],Pos[1],ContinentID,MapName_lang,Origin
0,Northshire Abbey,-8888.98,-0.5400,0,Eastern Kingdoms,TaxiNodes (Flight points table)
1,"Stormwind, Elwynn",-8841.06,489.6560,0,Eastern Kingdoms,TaxiNodes (Flight points table)
2,Programmer Isle,16391.81,16341.2100,0,Eastern Kingdoms,TaxiNodes (Flight points table)
3,"Sentinel Hill, Westfall",-10551.90,1034.3900,0,Eastern Kingdoms,TaxiNodes (Flight points table)
4,"Lakeshire, Redridge",-9429.10,-2231.4000,0,Eastern Kingdoms,TaxiNodes (Flight points table)
...,...,...,...,...,...,...
1196,Rondure Alcove,-3380.94,318.8450,2374,Zereth Mortis,TaxiNodes (Flight points table)
1197,Dormant Alcove,-3299.96,250.3440,2374,Zereth Mortis,TaxiNodes (Flight points table)
1198,Fulgor Alcove,-3374.48,112.9530,2374,Zereth Mortis,TaxiNodes (Flight points table)
1199,Arcae Locus,-3195.03,71.7205,2374,Zereth Mortis,TaxiNodes (Flight points table)


In [32]:
pd.concat([AreaPOI_joined, WaypointNode_joined, TaxiNodes_joined]).to_csv("JackJackLocations.csv", index=False)